In [1]:
from sequential_labelling import Label_Dict
print(Label_Dict)

OSError: [WinError 1455] The paging file is too small for this operation to complete. Error loading "c:\.virtualenv\sequence-processing\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.